# Angular Velocity: Intuitive 3D Animations (No glTF airplane dependency)
Your error came from airplane meshes being **glTF 1.0** (trimesh supports glTF 2.0 only).

This notebook avoids that by using:
- a **procedural airplane mesh** (fuselage + wings + tail)
- a **real robot mesh** from MoveIt if `git` works; otherwise a procedural gripper mesh

Each demo shows:
- angular velocity vector \(\omega\)
- body axes
- a marked point (wing tip / fingertip) with tangential velocity \(v=\omega\times r\)


In [ ]:
import os, glob, math, subprocess, shutil
import numpy as np

import plotly.graph_objects as go
import trimesh

np.set_printoptions(precision=4, suppress=True)

## Helpers

In [ ]:
def unit(v, eps=1e-12):
    v = np.asarray(v, dtype=float).reshape(-1)
    n = np.linalg.norm(v)
    return v if n < eps else v/n

def skew(w):
    w1,w2,w3 = np.asarray(w, dtype=float).reshape(3,)
    return np.array([[0, -w3, w2],
                     [w3, 0, -w1],
                     [-w2, w1, 0]], dtype=float)

def rodrigues(axis_or_omega, theta):
    axis = unit(axis_or_omega)
    K = skew(axis)
    I = np.eye(3)
    return I + math.sin(theta)*K + (1-math.cos(theta))*(K@K)

def apply_R(R, V):
    return (R @ np.asarray(V).T).T

def mesh_to_plotly(mesh: trimesh.Trimesh, name="mesh", opacity=1.0):
    v = mesh.vertices
    f = mesh.faces
    return go.Mesh3d(
        x=v[:,0], y=v[:,1], z=v[:,2],
        i=f[:,0], j=f[:,1], k=f[:,2],
        name=name,
        opacity=opacity,
        flatshading=True,
        showscale=False,
    )

def axes_traces(R=np.eye(3), origin=(0,0,0), L=1.0):
    o = np.array(origin, float)
    ex, ey, ez = R[:,0], R[:,1], R[:,2]
    traces = []
    for vec in [ex, ey, ez]:
        p = o + L*vec
        traces.append(go.Scatter3d(
            x=[o[0], p[0]], y=[o[1], p[1]], z=[o[2], p[2]],
            mode="lines",
            line=dict(width=7),
            showlegend=False
        ))
    return traces

def arrow_trace(vec, origin=(0,0,0), name="arrow", scale=1.0, width=10):
    o = np.array(origin, float)
    v = np.array(vec, float)*scale
    p = o + v
    return go.Scatter3d(
        x=[o[0], p[0]], y=[o[1], p[1]], z=[o[2], p[2]],
        mode="lines+markers",
        name=name,
        marker=dict(size=3),
        line=dict(width=width),
        showlegend=False
    )

def set_scene(fig, lim=2.0, title=""):
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis=dict(range=[-lim, lim]),
            yaxis=dict(range=[-lim, lim]),
            zaxis=dict(range=[-lim, lim]),
            aspectmode="cube"
        ),
        margin=dict(l=0, r=0, t=40, b=0)
    )
    return fig

def normalize_mesh(mesh: trimesh.Trimesh, target=1.0):
    if mesh is None: 
        return None
    mesh = mesh.copy()
    c = mesh.bounding_box.centroid
    mesh.apply_translation(-c)
    ext = mesh.extents
    s = (2*target) / (np.max(ext) + 1e-12)
    mesh.apply_scale(s)
    return mesh

## Build a simple airplane mesh (procedural)

In [ ]:
def make_airplane_mesh():
    # Fuselage along +x
    fus = trimesh.creation.cylinder(radius=0.12, height=2.0, sections=32)
    fus.apply_transform(trimesh.transformations.rotation_matrix(np.pi/2, [0,1,0]))

    nose = trimesh.creation.icosphere(subdivisions=2, radius=0.14)
    nose.apply_translation([1.05, 0.0, 0.0])

    wing = trimesh.creation.box(extents=[0.25, 1.3, 0.03])

    tail = trimesh.creation.box(extents=[0.18, 0.55, 0.02])
    tail.apply_translation([-0.85, 0.0, 0.02])

    fin = trimesh.creation.box(extents=[0.12, 0.04, 0.25])
    fin.apply_translation([-0.9, 0.0, 0.15])

    plane = trimesh.util.concatenate([fus, nose, wing, tail, fin])
    return normalize_mesh(plane, target=1.0)

air_mesh = make_airplane_mesh()
print("air mesh:", len(air_mesh.vertices), len(air_mesh.faces))

## Load a real robot mesh (MoveIt) if possible; otherwise fallback

In [ ]:
ASSETS_DIR = "assets_meshes"
os.makedirs(ASSETS_DIR, exist_ok=True)

def git_clone_if_needed(url, dst, depth=1):
    if os.path.exists(dst) and os.path.isdir(dst) and os.listdir(dst):
        return True
    try:
        if os.path.exists(dst):
            shutil.rmtree(dst)
        subprocess.check_call(["git", "clone", "--depth", str(depth), url, dst])
        return True
    except Exception as e:
        print("git clone failed:", e)
        return False

def load_mesh_any(path):
    if path is None:
        return None
    mesh_or_scene = trimesh.load(path, force="scene")
    if isinstance(mesh_or_scene, trimesh.Scene):
        mesh = trimesh.util.concatenate([g for g in mesh_or_scene.geometry.values()])
    else:
        mesh = mesh_or_scene
    mesh = mesh.copy()
    # These methods don't exist in all trimesh versions, so check before calling
    if hasattr(mesh, 'remove_unreferenced_vertices'):
        mesh.remove_unreferenced_vertices()
    if hasattr(mesh, 'remove_duplicate_faces'):
        mesh.remove_duplicate_faces()
    if hasattr(mesh, 'remove_degenerate_faces'):
        mesh.remove_degenerate_faces()
    return mesh

def make_simple_gripper():
    palm = trimesh.creation.box(extents=[0.35, 0.18, 0.12])
    finger1 = trimesh.creation.box(extents=[0.35, 0.05, 0.06])
    finger2 = trimesh.creation.box(extents=[0.35, 0.05, 0.06])
    finger1.apply_translation([0.15, 0.10, 0.00])
    finger2.apply_translation([0.15, -0.10, 0.00])
    g = trimesh.util.concatenate([palm, finger1, finger2])
    return normalize_mesh(g, target=0.9)

robot_repo = os.path.join(ASSETS_DIR, "moveit_resources")
ok = git_clone_if_needed("https://github.com/moveit/moveit_resources.git", robot_repo)

robot_mesh = None
if ok:
    robot_candidates = glob.glob(os.path.join(robot_repo, "**", "*hand*.stl"), recursive=True) + \
                       glob.glob(os.path.join(robot_repo, "**", "*gripper*.stl"), recursive=True) + \
                       glob.glob(os.path.join(robot_repo, "**", "*hand*.dae"), recursive=True)
    robot_candidates = sorted(robot_candidates, key=lambda p: (os.path.getsize(p), len(p)))
    robot_path = robot_candidates[0] if robot_candidates else None
    print("robot mesh path:", robot_path)
    if robot_path:
        robot_mesh = normalize_mesh(load_mesh_any(robot_path), target=0.9)

if robot_mesh is None:
    print("Falling back to procedural gripper mesh.")
    robot_mesh = make_simple_gripper()

print("robot mesh:", len(robot_mesh.vertices), len(robot_mesh.faces))

## Demo 1 — Basic 3D: tangential velocity \(\dot p = \omega \times p\)

In [ ]:
omega = np.array([0.2, 0.6, 1.0])   # rad/s
p0 = unit([1.0, 0.3, 0.0])

T = 4.0
N = 80
ts = np.linspace(0, T, N)

frames = []
for t in ts:
    R = rodrigues(omega, np.linalg.norm(omega)*t)
    p_t = R @ p0
    v_t = np.cross(omega, p_t)
    v_vis = 0.5 * v_t/(np.linalg.norm(v_t)+1e-12)

    data = []
    data += axes_traces(np.eye(3), L=1.2)
    data.append(arrow_trace(omega, name="ω", scale=0.5))
    data.append(arrow_trace(p_t, name="p(t)", scale=1.0, width=12))
    data.append(arrow_trace(v_vis, origin=p_t, name="tangent v", scale=1.0, width=8))
    frames.append(go.Frame(data=data, name=f"{t:.2f}"))

fig = go.Figure(frames=frames)
fig.add_traces(frames[0].data)
set_scene(fig, lim=1.6, title="Demo 1: p(t) and tangent v = ω × p")

fig.update_layout(
    updatemenus=[dict(type="buttons",
        buttons=[
            dict(label="Play", method="animate", args=[None, {"frame": {"duration": 40, "redraw": True}, "fromcurrent": True}]),
            dict(label="Pause", method="animate", args=[[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate"}]),
        ],
        x=0.02, y=0.02
    )],
    sliders=[dict(
        steps=[dict(method="animate", args=[[f.name], {"mode":"immediate", "frame":{"duration":0,"redraw":True}}], label=f.name) for f in frames],
        x=0.1, y=0.02, len=0.85
    )]
)
fig

## Demo 2 — Airplane: yaw/pitch/roll + wing-tip velocity
We estimate \(\omega\) from \(R(t)\) using \(\omega^\wedge \approx \dot R R^\top\), then draw \(v=\omega\times r\).

In [ ]:
V = air_mesh.vertices
wing_tip = V[np.argmax(V[:,1])]

T = 6.0
N = 90
ts = np.linspace(0, T, N)

yaw = np.deg2rad(40)*np.sin(2*np.pi*ts/T)
pitch = np.deg2rad(20)*np.sin(2*np.pi*ts/T + 0.6)
roll = np.deg2rad(30)*np.sin(2*np.pi*ts/T + 1.2)

Rs = []
for i in range(N):
    R = (rodrigues([0,0,1], yaw[i]) @
         rodrigues([0,1,0], pitch[i]) @
         rodrigues([1,0,0], roll[i]))
    Rs.append(R)
Rs = np.array(Rs)

dt = ts[1]-ts[0]
frames = []
for i,t in enumerate(ts):
    R = Rs[i]
    Vt = apply_R(R, air_mesh.vertices)
    mesh_t = trimesh.Trimesh(vertices=Vt, faces=air_mesh.faces, process=False)

    if 0 < i < N-1:
        Rdot = (Rs[i+1]-Rs[i-1])/(2*dt)
    elif i == 0:
        Rdot = (Rs[1]-Rs[0])/dt
    else:
        Rdot = (Rs[-1]-Rs[-2])/dt

    omega_hat = Rdot @ R.T
    omega_s = np.array([omega_hat[2,1], omega_hat[0,2], omega_hat[1,0]])

    pt = R @ wing_tip
    vt = np.cross(omega_s, pt)
    vt_vis = 0.5 * vt/(np.linalg.norm(vt)+1e-12)

    data = []
    data.append(mesh_to_plotly(mesh_t, name="airplane", opacity=0.95))
    data += axes_traces(np.eye(3), L=1.2)   # space axes
    data += axes_traces(R, L=1.0)           # body axes
    data.append(arrow_trace(omega_s, name="ω", scale=0.45, width=10))
    data.append(go.Scatter3d(x=[pt[0]], y=[pt[1]], z=[pt[2]], mode="markers", marker=dict(size=5), showlegend=False))
    data.append(arrow_trace(vt_vis, origin=pt, name="v", scale=1.0, width=8))

    frames.append(go.Frame(data=data, name=f"{t:.2f}"))

fig = go.Figure(frames=frames)
fig.add_traces(frames[0].data)
set_scene(fig, lim=2.0, title="Demo 2: Airplane yaw/pitch/roll + v = ω × r (wing tip)")

fig.update_layout(
    updatemenus=[dict(type="buttons",
        buttons=[
            dict(label="Play", method="animate", args=[None, {"frame": {"duration": 45, "redraw": True}, "fromcurrent": True}]),
            dict(label="Pause", method="animate", args=[[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate"}]),
        ],
        x=0.02, y=0.02
    )],
    sliders=[dict(
        steps=[dict(method="animate", args=[[f.name], {"mode":"immediate", "frame":{"duration":0,"redraw":True}}], label=f.name) for f in frames],
        x=0.1, y=0.02, len=0.85
    )]
)
fig

## Demo 2b — Pretty Airplane: yaw/pitch/roll + wing-tip velocity (using the pretty mesh)



In [ ]:
import trimesh
import numpy as np

def create_triangular_prism(base_points_2d, height, center=[0, 0, 0]):
    """
    Create a triangular prism by extruding a 2D triangle along Z.
    base_points_2d: array of shape (3, 2) with triangle vertices in XY plane
    height: extrusion height along Z
    """
    base_points_2d = np.asarray(base_points_2d)
    # Create bottom and top faces
    bottom = np.column_stack([base_points_2d, np.zeros(3)])  # z=0
    top = np.column_stack([base_points_2d, np.full(3, height)])  # z=height
    
    # Vertices: bottom triangle (0,1,2) + top triangle (3,4,5)
    vertices = np.vstack([bottom, top])
    
    # Faces: 2 triangle faces (bottom, top) + 3 rectangular sides
    faces = np.array([
        # Bottom face
        [0, 2, 1],
        # Top face
        [3, 4, 5],
        # Side faces (rectangles as 2 triangles each)
        [0, 1, 3], [1, 4, 3],
        [1, 2, 4], [2, 5, 4],
        [2, 0, 5], [0, 3, 5],
    ], dtype=int)
    
    mesh = trimesh.Trimesh(vertices=vertices, faces=faces)
    if center != [0, 0, 0]:
        mesh.apply_translation(center)
    return mesh

def make_airplane_mesh_pretty():
    parts = []

    # Fuselage: capsule (rounded ends) looks nicer than a cylinder
    fus = trimesh.creation.capsule(radius=0.10, height=1.8, count=[24, 24])
    # capsule is along Z; rotate to X
    fus.apply_transform(trimesh.transformations.rotation_matrix(np.pi/2, [0,1,0]))
    parts.append(fus)

    # Nose cone
    nose = trimesh.creation.cone(radius=0.12, height=0.25, sections=32)
    nose.apply_transform(trimesh.transformations.rotation_matrix(np.pi/2, [0,1,0]))
    nose.apply_translation([1.0, 0, 0])
    parts.append(nose)

    # Main wing: triangular prism
    wing_poly = np.array([[0.15, 0.0],
                          [-0.15, 0.55],
                          [-0.15, -0.55]])
    # Create prism with thin thickness in Z (0.02), then translate
    wing = create_triangular_prism(wing_poly, height=0.02)
    wing.apply_translation([0.1, 0, -0.01])  # center in Z
    parts.append(wing)

    # Tail wing (smaller triangular prism)
    tail_poly = np.array([[0.08, 0.0],
                          [-0.10, 0.25],
                          [-0.10, -0.25]])
    tail = create_triangular_prism(tail_poly, height=0.015)
    tail.apply_translation([-0.75, 0, 0.0125])  # center in Z
    parts.append(tail)

    # Vertical fin
    fin = trimesh.creation.box(extents=[0.20, 0.02, 0.22])
    fin.apply_translation([-0.78, 0, 0.14])
    parts.append(fin)

    # Propeller (simple cross)
    prop1 = trimesh.creation.box(extents=[0.02, 0.35, 0.01])
    prop2 = trimesh.creation.box(extents=[0.02, 0.35, 0.01])
    prop2.apply_transform(trimesh.transformations.rotation_matrix(np.pi/2, [1,0,0]))
    prop = trimesh.util.concatenate([prop1, prop2])
    prop.apply_translation([1.05, 0, 0])
    parts.append(prop)

    plane = trimesh.util.concatenate(parts)
    return normalize_mesh(plane, target=1.0)

air_mesh = make_airplane_mesh_pretty()


In [ ]:
V = air_mesh.vertices
wing_tip = V[np.argmax(V[:,1])]

T = 6.0
N = 90
ts = np.linspace(0, T, N)

yaw = np.deg2rad(40)*np.sin(2*np.pi*ts/T)
pitch = np.deg2rad(20)*np.sin(2*np.pi*ts/T + 0.6)
roll = np.deg2rad(30)*np.sin(2*np.pi*ts/T + 1.2)

Rs = []
for i in range(N):
    R = (rodrigues([0,0,1], yaw[i]) @
         rodrigues([0,1,0], pitch[i]) @
         rodrigues([1,0,0], roll[i]))
    Rs.append(R)
Rs = np.array(Rs)

dt = ts[1]-ts[0]
frames = []
for i,t in enumerate(ts):
    R = Rs[i]
    Vt = apply_R(R, air_mesh.vertices)
    mesh_t = trimesh.Trimesh(vertices=Vt, faces=air_mesh.faces, process=False)

    if 0 < i < N-1:
        Rdot = (Rs[i+1]-Rs[i-1])/(2*dt)
    elif i == 0:
        Rdot = (Rs[1]-Rs[0])/dt
    else:
        Rdot = (Rs[-1]-Rs[-2])/dt

    omega_hat = Rdot @ R.T
    omega_s = np.array([omega_hat[2,1], omega_hat[0,2], omega_hat[1,0]])

    pt = R @ wing_tip
    vt = np.cross(omega_s, pt)
    vt_vis = 0.5 * vt/(np.linalg.norm(vt)+1e-12)

    data = []
    data.append(mesh_to_plotly(mesh_t, name="airplane", opacity=0.95))
    data += axes_traces(np.eye(3), L=1.2)   # space axes
    data += axes_traces(R, L=1.0)           # body axes
    data.append(arrow_trace(omega_s, name="ω", scale=0.45, width=10))
    data.append(go.Scatter3d(x=[pt[0]], y=[pt[1]], z=[pt[2]], mode="markers", marker=dict(size=5), showlegend=False))
    data.append(arrow_trace(vt_vis, origin=pt, name="v", scale=1.0, width=8))

    frames.append(go.Frame(data=data, name=f"{t:.2f}"))

fig = go.Figure(frames=frames)
fig.add_traces(frames[0].data)
set_scene(fig, lim=2.0, title="Demo 2b: Pretty Airplane yaw/pitch/roll + v = ω × r (wing tip)")

fig.update_layout(
    updatemenus=[dict(type="buttons",
        buttons=[
            dict(label="Play", method="animate", args=[None, {"frame": {"duration": 45, "redraw": True}, "fromcurrent": True}]),
            dict(label="Pause", method="animate", args=[[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate"}]),
        ],
        x=0.02, y=0.02
    )],
    sliders=[dict(
        steps=[dict(method="animate", args=[[f.name], {"mode":"immediate", "frame":{"duration":0,"redraw":True}}], label=f.name) for f in frames],
        x=0.1, y=0.02, len=0.85
    )]
)
fig



airplane 3d transformation video

## Demo 3 — Robot mesh: spin + fingertip velocity

In [ ]:
V = robot_mesh.vertices
tip = V[np.argmax(V[:,0])]

omega = np.array([0.0, 1.2, 0.6])  # rad/s

T = 4.0
N = 80
ts = np.linspace(0, T, N)

frames = []
for t in ts:
    theta = np.linalg.norm(omega)*t
    R = rodrigues(omega, theta)

    Vt = apply_R(R, robot_mesh.vertices)
    mesh_t = trimesh.Trimesh(vertices=Vt, faces=robot_mesh.faces, process=False)

    pt = R @ tip
    vt = np.cross(omega, pt)
    vt_vis = 0.45 * vt/(np.linalg.norm(vt)+1e-12)

    data = []
    data.append(mesh_to_plotly(mesh_t, name="robot", opacity=0.95))
    data += axes_traces(np.eye(3), L=1.2)
    data += axes_traces(R, L=1.0)
    data.append(arrow_trace(omega, name="ω", scale=0.45, width=10))
    data.append(go.Scatter3d(x=[pt[0]], y=[pt[1]], z=[pt[2]], mode="markers", marker=dict(size=5), showlegend=False))
    data.append(arrow_trace(vt_vis, origin=pt, name="v", scale=1.0, width=8))

    frames.append(go.Frame(data=data, name=f"{t:.2f}"))

fig = go.Figure(frames=frames)
fig.add_traces(frames[0].data)
set_scene(fig, lim=1.8, title="Demo 3: Robot mesh spin + v = ω × r (tip)")

fig.update_layout(
    updatemenus=[dict(type="buttons",
        buttons=[
            dict(label="Play", method="animate", args=[None, {"frame": {"duration": 45, "redraw": True}, "fromcurrent": True}]),
            dict(label="Pause", method="animate", args=[[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate"}]),
        ],
        x=0.02, y=0.02
    )],
    sliders=[dict(
        steps=[dict(method="animate", args=[[f.name], {"mode":"immediate", "frame":{"duration":0,"redraw":True}}], label=f.name) for f in frames],
        x=0.1, y=0.02, len=0.85
    )]
)
fig